In [ ]:
import numpy as np
import pandas as pd
from functools import partial
from pprint import pprint
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>div.container { width:100% !important; }</style>"))
display(HTML("<style>div.cell { width:65% !important; margin: auto; }</style>"))
display(HTML("<style>div.output_png { background-color: white; position: relative; width: 400% !important; left: -35%; overflow-x: visible !important; }</style>"))
display(HTML("<style>div.output_png img { max-width: 1900px !important; margin: auto !important; }</style>"))

%load_ext autoreload
%autoreload 1

import sys
import os.path
absolute_path_code_dir = os.path.join('/', 'home', 'justine', 'Dropbox', 'imperial', 'computing', 'thesis', 'code')
sys.path.insert(0, absolute_path_code_dir)

from helpers.i_o import join_repetitions
import analyse.visualise_adaptive as vis
from analyse.regret import transform_for_plot as add_regret

# Convex combination of two adaptive filters

### load and prepare data

In [ ]:
BATCH_IDs = ['', '']
DATASET_DIR = os.path.join('.')

DF = join_repetitions(os.path.join(DATASET_DIR, '%s-df.dat' % batch_id) for batch_id in BATCH_IDs)

total_timesteps = DF.columns.get_level_values('timestep')[-1]
specs           = vis.get_batch_specs(BATCH_IDs[0], DATASET_DIR)  ## assumes specs are the same
attacks         = vis.get_attacks_info_from_specs(specs, N=total_timesteps)

pprint(attacks)
print('\ntotal timesteps: %d' % total_timesteps)

DF = DF.mean(level=DF.index.names[:-1])
DF['join_id'] = ['join %d' % num for num in range(len(DF))]
DF = DF.set_index('join_id', append=True)
DF.index = DF.index.rename('experiment_id', level=-1)

BATCH_ID = 'join'
experiment_ids = vis.get_sorted_experiment_ids(DF)

display(DF)

In [ ]:
df = (DF.xs('adaline', axis=0, level='classifier fast')
        .xs(0.005, axis=0, level='η fast')
        .xs(0.0005, axis=0, level='η slow')
        .xs('dictionary', axis=0, level='attack')
        .xs(0.5, axis=0, level='adaptation rate'))
experiment_ids = vis.get_sorted_experiment_ids(df)

for experiment_ids_group in vis.chunks(experiment_ids, groups_of=2):
    vis.compare_group(DF, experiment_ids_group, plot_specs='test set', attacks_info=attacks)
    vis.compare_group(DF, experiment_ids_group, plot_specs='regret',   attacks_info=attacks, transform=partial(add_regret, T=500))


custom_params = {'xlim': (19000, 20500), 'ylim': (0, 200), 'figsize': (27, 14)}

for experiment_ids_group in vis.chunks(experiment_ids, groups_of=2):
    vis.compare_group(DF, experiment_ids_group, plot_specs='regret', attacks_info=attacks, transform=partial(add_regret, T=1000),
                  **custom_params)

## Observations and early interpretation

- observation